In [1]:
import os

import pandas as pd
from pycaret import regression


# Load Nagesh's dataset


In [2]:
os.makedirs("./results/stats", exist_ok=True)

df = pd.read_csv(f"./datasets/suanfarma_train_data.csv", index_col=0)
df.drop(["id"], axis=1, inplace=True)
df

,result,101WI610_mean,101WI610_std,101WI610_min,101WI610_25%,101WI610_50%,101WI610_75%,101WI610_max,306LI606_mean,306LI606_std,306LI606_min,306LI606_25%,306LI606_50%,306LI606_75%,306LI606_max,101AI635_mean,101AI635_std,101AI635_min,101AI635_25%,101AI635_50%,101AI635_75%,101AI635_max,101AI605_mean,101AI605_std,101AI605_min,101AI605_25%,101AI605_50%,101AI605_75%,101AI605_max,101TI607_mean,101TI607_std,101TI607_min,101TI607_25%,101TI607_50%,101TI607_75%,101TI607_max,101TI637_mean,101TI637_std,101TI637_min,101TI637_25%,101TI637_50%,101TI637_75%,101TI637_max,306TI604_mean,306TI604_std,306TI604_min,306TI604_25%,306TI604_50%,306TI604_75%,306TI604_max,158PI678_mean,158PI678_std,158PI678_min,158PI678_25%,158PI678_50%,158PI678_75%,158PI678_max,101PI602_mean,101PI602_std,101PI602_min,101PI602_25%,101PI602_50%,101PI602_75%,101PI602_max,101PI603_mean,101PI603_std,101PI603_min,101PI603_25%,101PI603_50%,101PI603_75%,101PI603_max,158PIC678_823_mean,158PIC678_823_std,158PIC678_823_min,158PIC678_823_25%,158PIC678_823_50%,158PIC678_823_75%,158PIC678_823_max,158PIC678_824_mean,158PIC678_824_std,158PIC678_824_min,158PIC678_824_25%,158PIC678_824_50%,158PIC678_824_75%,158PIC678_824_max,99PI621_mean,99PI621_std,99PI621_min,99PI621_25%,99PI621_50%,99PI621_75%,99PI621_max,158FI652_mean,158FI652_std,158FI652_min,158FI652_25%,158FI652_50%,158FI652_75%,158FI652_max,158FI654_mean,158FI654_std,158FI654_min,158FI654_25%,158FI654_50%,158FI654_75%,158FI654_max,158FI656_mean,158FI656_std,158FI656_min,158FI656_25%,158FI656_50%,158FI656_75%,158FI656_max,158FI666_mean,158FI666_std,158FI666_min,158FI666_25%,158FI666_50%,158FI666_75%,158FI666_max,158FI667_mean,158FI667_std,158FI667_min,158FI667_25%,158FI667_50%,158FI667_75%,158FI667_max,158FI679P_mean,158FI679P_std,158FI679P_min,158FI679P_25%,158FI679P_50%,158FI679P_75%,158FI679P_max,158TI650_mean,158TI650_std,158TI650_min,158TI650_25%,158TI650_50%,158TI650_75%,158TI650_max,159FI652_mean,159FI652_std,159FI652_min,159FI652_25%,159FI652_50%,159FI652_75%,159FI652_max,159FI654_mean,159FI654_std,159FI654_min,159FI654_25%,159FI654_50%,159FI654_75%,159FI654_max,159FI656_mean,159FI656_std,159FI656_min,159FI656_25%,159FI656_50%,159FI656_75%,159FI656_max,159FI666_mean,159FI666_std,159FI666_min,159FI666_25%,159FI666_50%,159FI666_75%,159FI666_max,159FI667_mean,159FI667_std,159FI667_min,159FI667_25%,159FI667_50%,159FI667_75%,159FI667_max,159TI650_mean,159TI650_std,159TI650_min,159TI650_25%,159TI650_50%,159TI650_75%,159TI650_max,160FI652_mean,160FI652_std,160FI652_min,160FI652_25%,160FI652_50%,160FI652_75%,160FI652_max,160FI654_mean,160FI654_std,160FI654_min,160FI654_25%,160FI654_50%,160FI654_75%,160FI654_max,160FI656_mean,160FI656_std,160FI656_min,160FI656_25%,160FI656_50%,160FI656_75%,160FI656_max,160FI666_mean,160FI666_std,160FI666_min,160FI666_25%,160FI666_50%,160FI666_75%,160FI666_max,160FI667_mean,160FI667_std,160FI667_min,160FI667_25%,160FI667_50%,160FI667_75%,160FI667_max,160TI650_mean,160TI650_std,160TI650_min,160TI650_25%,160TI650_50%,160TI650_75%,160TI650_max,161FI652_mean,161FI652_std,161FI652_min,161FI652_25%,161FI652_50%,161FI652_75%,161FI652_max,161FI654_mean,161FI654_std,161FI654_min,161FI654_25%,161FI654_50%,161FI654_75%,161FI654_max,161FI656_mean,161FI656_std,161FI656_min,161FI656_25%,161FI656_50%,161FI656_75%,161FI656_max,161FI666_mean,161FI666_std,161FI666_min,161FI666_25%,...,161FI656_90,161FI656_95,161FI656_99,161FI666_0,161FI666_10,161FI666_100,161FI666_20,161FI666_25,161FI666_5,161FI666_50,161FI666_75,161FI666_80,161FI666_85,161FI666_90,161FI666_95,161FI666_99,161FI667_0,161FI667_10,161FI667_100,161FI667_20,161FI667_25,161FI667_5,161FI667_50,161FI667_75,161FI667_80,161FI667_85,161FI667_90,161FI667_95,161FI667_99,161TI650_0,161TI650_10,161TI650_100,161TI650_20,161TI650_25,161TI650_5,161TI650_50,161TI650_75,161TI650_80,161TI650_85,161TI650_90,161TI650_95,161TI650_99,162CAL4551_0,162CAL4551_10,162CAL4551_100,162CAL4551_20,162CAL4551_25,162CAL4551_5,162CAL4551_50,162CAL4551_75,162CAL4551_80,162CAL4551_85,162CA

# Setup PyCaret


In [3]:
setup = regression.setup(
    data=df,
    target="result",
    train_size=0.8,
    session_id=666,
    fold=2,
)


,Description,Value
0,session_id,666
1,Target,result
2,Original Data,"(38, 2081)"
3,Missing Values,False
4,Numeric Features,2080
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(30, 1884)"


# Train some models

In [4]:
models = regression.compare_models(
    include=regression.models().index.tolist(), n_select=100
)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,0.0197,0.0006,0.0247,0.1749,0.0141,0.0265,0.0750
et,Extra Trees Regressor,0.0200,0.0006,0.0252,0.1301,0.0144,0.0268,0.4750
rf,Random Forest Regressor,0.0215,0.0007,0.0261,0.0758,0.0150,0.0289,0.4500
catboost,CatBoost Regressor,0.0212,0.0007,0.0264,0.0507,0.0151,0.0285,29.3500
ada,AdaBoost Regressor,0.0213,0.0007,0.0267,0.0387,0.0153,0.0286,0.1350
xgboost,Extreme Gradient Boosting,0.0225,0.0007,0.0264,0.0257,0.0151,0.0303,0.5150
gbr,Gradient Boosting Regressor,0.0228,0.0007,0.0267,0.0141,0.0153,0.0304,0.2250
lightgbm,Light Gradient Boosting Machine,0.0218,0.0007,0.0271,-0.0002,0.0155,0.0294,0.0700
llar,Lasso Least Angle Regression,0.0218,0.0007,0.0271,-0.0002,0.0155,0.0294,0.0200
svm,Support Vector Regression,0.0224,0.0008,0.0277,-0.0530,0.0159,0.0300,0.0200


# Get features in order of importance for first five models


In [64]:
df_features = pd.DataFrame()

for model in models[:5]:
    tmp = None
    try:
        tmp = pd.DataFrame(
            {
                "Feature": regression.get_config("X_train").columns,
                "Value": abs(model.coef_[0]),
            }
        ).sort_values(by="Value", ascending=False)
    except Exception as ex:
        print(ex)
        try:
            tmp = pd.DataFrame(
                {
                    "Feature": regression.get_config("X_train").columns,
                    "Value": abs(model.feature_importances_[0]),
                }
            ).sort_values(by="Value", ascending=False)
        except Exception as ex:
            print(ex)
    if isinstance(tmp, pd.DataFrame):
        tmp.reset_index(drop=True, inplace=True)
        tmp["Model"] = str(type(model))
        df_features = df_features.append(tmp)
df_features.reset_index(inplace=True)
df_features.to_csv("./results/feature_importances.csv", index=False)
df_features


'KNeighborsRegressor' object has no attribute 'coef_'
'KNeighborsRegressor' object has no attribute 'feature_importances_'
'ExtraTreesRegressor' object has no attribute 'coef_'
'RandomForestRegressor' object has no attribute 'coef_'
'CatBoostRegressor' object has no attribute 'coef_'
'AdaBoostRegressor' object has no attribute 'coef_'


,index,Feature,Value,Model
0,0,101WI610_mean,2.246859e-07,<class 'sklearn.ensemble._forest.ExtraTreesReg...
1,1,118TI970_99,2.246859e-07,<class 'sklearn.ensemble._forest.ExtraTreesReg...
2,2,118TI971_95,2.246859e-07,<class 'sklearn.ensemble._forest.ExtraTreesReg...
3,3,118TI971_90,2.246859e-07,<class 'sklearn.ensemble._forest.ExtraTreesReg...
4,4,118TI971_85,2.246859e-07,<class 'sklearn.ensemble._forest.ExtraTreesReg...
...,...,...,...,...
7531,1879,107AI672_50%,0.000000e+00,<class 'sklearn.ensemble._weight_boosting.AdaB...
7532,1880,107AI672_25%,0.000000e+00,<class 'sklearn.ensemble._weight_boosting.AdaB...
7533,1881,107AI672_min,0.000000e+00,<class 'sklearn.ensemble._weight_boosting.AdaB...
7534,1882,107AI672_std,0.000000e+00,<class 'sklearn.ensemble._weight_boosting.AdaB...


# Groupby feature (sum the indices) and sort them


In [65]:
df_features = pd.read_csv("./results/feature_importances.csv")
df_mean_feature_importance = pd.DataFrame(
    df_features.groupby(by=["Feature"])["index"].sum().sort_values(ascending=True)
)
df_mean_feature_importance.to_csv("./results/mean_feature_importances.csv")
df_mean_feature_importance


,index
Feature,
101WI610_mean,0
118TI970_99,4
118TI971_95,8
118TI971_90,12
118TI971_85,16
...,...
107AI672_50%,7516
107AI672_25%,7520
107AI672_min,7524


# Get actual features


In [70]:
df_mean_feature_importance = pd.read_csv("./results/mean_feature_importances.csv")
df_mean_feature_importance[["Sensor", "IDK"]] = df_mean_feature_importance[
    "Feature"
].str.split("_", 1, expand=True)
df_best_features = df_mean_feature_importance.drop_duplicates("Sensor")["Sensor"]
df_best_features.to_csv("./results/best_features.csv", index=False)
df_best_features.tolist()


['101WI610',
 '118TI970',
 '118TI971',
 '118SI613',
 '118TI978',
 '158CAL4552',
 '158CAL4551',
 '118PI950',
 '118PI954',
 '118PI926',
 '118PI964',
 '158FI679P',
 '158PI678',
 '158FI667',
 '158PIC678',
 '158TI650',
 '158FI652',
 '158FI654',
 '158FI656',
 '158FI666',
 '118EI630',
 '118EI640',
 '118EI620',
 '118FI602',
 '118FI913',
 '118FI983',
 '118AI631',
 '118AI627',
 '118AI641',
 '118EI610',
 '118PI914',
 '118PI916',
 '118PI915',
 '118PI649',
 '118PI925',
 '118PI924',
 '118PI618',
 '118PI619',
 '118PI638',
 '162CAL4551',
 '162FI656',
 '162FI654',
 '161TI650',
 '162FI666',
 '163FI654',
 '162TI650',
 '161FI654',
 '161FI656',
 '161FI652',
 '161FI666',
 '161FI667',
 '306LI606',
 '306TI604',
 '165TI650',
 'CONCTOTALE',
 '99PI621',
 '165CAL4551',
 '163FI666',
 '164CAL4551',
 '163FI656',
 '164FI654',
 '159FI656',
 '159FI666',
 '159FI654',
 '159TI650',
 '159FI667',
 '158TI672',
 '158TI684',
 '158TI673',
 '159FI652',
 '159CAL4551',
 '160FI666',
 '160FI667',
 '160CAL4551',
 '160FI656',
 '160TI6